In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pytz import timezone
import datetime
import time

from pyspark.sql.functions import *
import ibmos2spark, os
spark = SparkSession.builder.getOrCreate()

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210530220221-0005
KERNEL_ID = 5d70bb58-a33d-493f-a195-6903775c1b50


In [2]:
if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_70e1c01a14bc45d18469c408d82a5ca4 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_70e1c01a14bc45d18469c408d82a5ca4 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

credentials = {
    'endpoint': endpoint_70e1c01a14bc45d18469c408d82a5ca4,
    'service_id': 'iam-ServiceId-f1901189-ea90-4d93-9035-0f6aa227c06a',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'NbPeUpIdUdV_UiY2wkFWSV56iiOcq3eS_estY6iYcK0c'
}

configuration_name = 'os_70e1c01a14bc45d18469c408d82a5ca4_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

dfvisaolojistavendanps = spark.read\
  .format('parquet')\
  .option('header', 'true')\
  .load(cos.url('visaolojistavendanps.snappy.parquet', 'projetoviagrupo4-donotdelete-pr-ktlzuttu142hda'))

print('Carregado Parquet de Visão Lojistica Venda NPS: {} linhas'.format(dfvisaolojistavendanps.count()))

dfcompraentrega = spark.read\
  .format('parquet')\
  .option('header', 'true')\
  .load(cos.url('compraentrega.snappy.parquet', 'projetoviagrupo4-donotdelete-pr-ktlzuttu142hda'))

print('Carregado Parquet de Compra Entrega: {} linhas'.format(dfcompraentrega.count()))

Carregado Parquet de Visão Lojistica Venda NPS: 174903 linhas
Carregado Parquet de Compra Entrega: 1723966 linhas


In [9]:
atraso_por_logista = dfcompraentrega.withColumn("dataentregaInter", dayofyear(col("dataentrega")))\
                       .withColumn("dataprometidaoriginalInt", dayofyear(col("dataprometidaoriginal")))\
                       .withColumn("atraso", (col('dataentregaInter') - col('dataprometidaoriginalInt')))\
                       .select('idlojista', 'atraso')


In [14]:
atraso_por_logista_agrupados = atraso_por_logista.groupBy(col('IdLojista'))\
                                                 .agg(avg('atraso').alias('media_atraso'))


In [11]:
nps_por_lojista = dfvisaolojistavendanps.groupBy(col('IdLojista'))\
.agg(sum('QtResposta').alias('qtresposta'),\
	sum('QtRespostaDetratora').alias('qtrespostadetratora'),\
	sum('QtRespostaNeutra').alias('qtrespostaneutra'),\
	sum('QtRespostaPromotora').alias('qtrespostapromotora'))\
.withColumn("nps_atualizado", (((col("qtrespostapromotora")*100) - (col("qtrespostadetratora")*100))/col('qtresposta')).cast(IntegerType()))

nps_por_lojista.show()

+---------+----------+-------------------+----------------+-------------------+--------------+
|IdLojista|qtresposta|qtrespostadetratora|qtrespostaneutra|qtrespostapromotora|nps_atualizado|
+---------+----------+-------------------+----------------+-------------------+--------------+
|    21700|        11|                  1|               1|                  9|            72|
|    33717|       124|                 29|              18|                 77|            38|
|    31528|        39|                  4|               6|                 29|            64|
|    26708|         8|                  1|               0|                  7|            75|
|    37307|         3|                  0|               0|                  3|           100|
|    32414|         5|                  2|               1|                  2|             0|
|    19079|        17|                  2|               4|                 11|            52|
|    34061|         3|                  1|        

In [18]:
nps_por_lojista_atualizado = nps_por_lojista.join(atraso_por_logista_agrupados, ['idlojista'], 'inner').orderBy('idlojista')

In [19]:
nps_por_lojista_atualizado.show(20)

+---------+----------+-------------------+----------------+-------------------+--------------+--------------------+
|IdLojista|qtresposta|qtrespostadetratora|qtrespostaneutra|qtrespostapromotora|nps_atualizado|        media_atraso|
+---------+----------+-------------------+----------------+-------------------+--------------+--------------------+
|    10012|        62|                  6|              10|                 46|            64| -3.2212389380530975|
|    10047|        31|                  8|               2|                 21|            41| 0.34959349593495936|
|    10055|       127|                  9|              17|                101|            72|   6.287671232876712|
|    10062|        71|                  8|              11|                 52|            61|   -9.83815028901734|
|    10065|         4|                  0|               0|                  4|           100|  -16.94736842105263|
|    10068|        85|                 11|              11|             

In [22]:
import os
from pyspark.sql import SparkSession
sparkSession = SparkSession(spark).builder.getOrCreate()


viahacka_db2_url = 'jdbc:db2://{}:{}/{}:sslConnection=true;'.format(
    'dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net',
    50001,
    'BLUDB'
)


nps_por_lojista_atualizado.write.format("jdbc")\
    .option("driver", "com.ibm.db2.jcc.DB2Driver")\
    .option("url", viahacka_db2_url) \
    .option("user", "xrz69130") \
    .option("password", """44k8g5m12^1sg0sc""") \
    .option("dbtable", "XRZ69130.nps_por_lojista_atualizado") \
    .mode("overwrite").save()